# Study note: Community detection or Principal Component Analysis - Different apporaches to analyze big datasets
_[Christopher Lux](https://github.com/LibraChris)_, Jan 2022

## Content
- [Introduction](#Introduction)
- [Loading the previous graph](#Loading-the-previous-graph)
- [Modularity](#Modularity)
- [PCA - Principal component analysis](#PCA-Principal-component-analysis)
- [Ontology Enrichment](#Ontology-Enrichment)
- [Further reading](#Further-reading)


## Introduction

This tutorial picks up where the Fslab advanced tutorials [Correlation network](https://fslab.org/content/tutorials/009_correlation-network.html) ended.
 The goal of this blogpost is to showcase different methods to analyze big datasets:
The graph-Analysis apporach featuring community detection and ontology enrichment vs 
the principal component analysis approach that reduces the dimensions of complex datasets.

We are using the following libraries:

1. [Deedle](https://github.com/fslaborg/Deedle)
2. [Plotly.NET](https://github.com/plotly/Plotly.NET/)
3. [Cyjs.NET](https://github.com/fslaborg/Cyjs.NET)
4. [BioFSharp](https://github.com/CSBiology/BioFSharp)
5. [FSharp.Data](https://github.com/fsprojects/FSharp.Data)
6. [FSharp.Stats](https://github.com/fslaborg/FSharp.Stats)
7. [FSharp.FGL](https://github.com/CSBiology/FSharp.FGL)

## Loading the previous graph 

We start by recreating the graph depicted in [Correlation network](https://fslab.org/content/tutorials/009_correlation-network.html) .

In [40]:
#r "nuget: FSharp.Data, 4.2.7"
#r "nuget: Deedle, 2.5.0"
#r "nuget: FSharp.Stats, 0.4.3"
#r "nuget: Cyjs.NET, 0.0.4"

open Deedle
open FSharp.Data
open FSharp.Stats
open FSharp.Stats.Testing
open Cyjs.NET

// Load the data 
let rawData = Http.RequestString @"https://raw.githubusercontent.com/HLWeil/datasets/main/data/ecoliGeneExpression.tsv"

// Create a deedle frame and index the rows with the values of the "Key" column.
let rawFrame : Frame<string,string> = 
    Frame.ReadCsvString(rawData, separators = "\t")
    |> Frame.take 500
    |> Frame.indexRows "Key"

// Get the rows as a matrix
let rows = 
    rawFrame 
    |> Frame.toJaggedArray 
    |> Matrix.ofJaggedArray

// Create a correlation network by computing the pearson correlation between every two rows
let correlationNetwork = 
    Correlation.Matrix.rowWisePearson rows

let thr = 0.8203125

// Set all correlations less strong than the critical threshold to 0
let filteredNetwork = 
    correlationNetwork
    |> Matrix.map (fun v -> if (abs v) > thr then v else 0.)

// The styled vertices. The size is based on the degree of this vertex, so that more heavily connected nodes are emphasized
let cytoVertices = 
    rawFrame.RowKeys
    |> Seq.toList
    |> List.indexed
    |> List.choose (fun (i,v) -> 
        let degree = 
            Matrix.getRow filteredNetwork i 
            |> Seq.filter ((<>) 0.)
            |> Seq.length
        let styling = [CyParam.label v; CyParam.weight (sqrt (float degree) + 1. |> (*) 10.)]

        if degree > 1 then 
            Some (Elements.node (string i) styling)
        else 
            None
    )

// Styled edges
let cytoEdgesOG = 
    let len = filteredNetwork.Dimensions |> fst
    [
        for i = 0 to len - 1 do
            for j = i + 1 to len - 1 do
                let v = filteredNetwork.[i,j]
                if v <> 0. then yield i,j,v
    ]
    |> List.mapi (fun i (v1,v2,weight) -> 
        let styling = [CyParam.weight (0.2 * weight)]
        Elements.edge ("e" + string i) (string v1) (string v2) styling
    )
   
// Resulting cytograph
let cytoGraph = 

    CyGraph.initEmpty ()
    |> CyGraph.withElements cytoVertices
    |> CyGraph.withElements cytoEdgesOG
    |> CyGraph.withStyle "node" 
        [
            CyParam.shape "circle"
            CyParam.content =. CyParam.label
            CyParam.width =. CyParam.weight
            CyParam.height =. CyParam.weight
            CyParam.Text.Align.center
            CyParam.Border.color "#A00975"
            CyParam.Border.width 3
        ]
    |> CyGraph.withStyle "edge" 
        [
            CyParam.Line.color "#3D1244"
        ]
    |> CyGraph.withLayout (Layout.initCose (Layout.LayoutOptions.Cose(NodeOverlap = 400,ComponentSpacing = 100)))  

System.IO.File.ReadAllText "../../files/ecoliGeneExpressionCyjs.html"
|> DisplayFunctions.HTML

Installed Packages Cyjs.NET, 0.0.4 Deedle, 2.5.0 FSharp.Data, 4.2.7 FSharp.Stats, 0.4.3

<!DOCTYPE html>
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 

 <meta name="optimizely-datafile" content="{"groups": [], "environmentKey": "production", "rollouts": [], "typedAudiences": [], "projectId": "16737760170", "variables": [], "featureFlags": [], "experiments": [], "version": "4", "audiences": [{"conditions": "[\"or\", {\"match\": \"exact\", \"name\": \"$opt_dummy_attribute\", \"type\": \"custom_attribute\", \"value\": \"$opt_dummy_value\"}]", "id": "$opt_dummy_audience", "name": "Optimizely-Generated Audience for Backwards Compatibility"}], "anonymizeIP": true, "sdkKey": "WTc6awnGuYDdG98CYRban", "attributes": [{"id": "16822470375", "key": "user_id"}, {"id": "17143601254", "key": "spammy"}, {"id": "18175660309", "key": "organization_plan"}, {"id": "18813001570", "key": "is_logged_in"}, {"id": "19073851829", "key": "geo"}, {"id": "20175462351", "key": "requestedCurrency"}, {"id": "20785470195", "key": "country_code"}, {"id": "21656311196", "key": "opened_downgrade_dialog"}], "botFiltering": false, "accountId": "16737760170", "events": [{"experimentIds": [], "id": "17911811441", "key": "hydro_click.dashboard.teacher_toolbox_cta"}, {"experimentIds": [], "id": "18124116703", "key": "submit.organizations.complete_sign_up"}, {"experimentIds": [], "id": "18145892387", "key": "no_metric.tracked_outside_of_optimizely"}, {"experimentIds": [], "id": "18178755568", "key": "click.org_onboarding_checklist.add_repo"}, {"experimentIds": [], "id": "18180553241", "key": "submit.repository_imports.create"}, {"experimentIds": [], "id": "18186103728", "key": "click.help.learn_more_about_repository_creation"}, {"experimentIds": [], "id": "18188530140", "key": "test_event"}, {"experimentIds": [], "id": "18191963644", "key": "click.empty_org_repo_cta.transfer_repository"}, {"experimentIds": [], "id": "18195612788", "key": "click.empty_org_repo_cta.import_repository"}, {"experimentIds": [], "id": "18210945499", "key": "click.org_onboarding_checklist.invite_members"}, {"experimentIds": [], "id": "18211063248", "key": "click.empty_org_repo_cta.create_repository"}, {"experimentIds": [], "id": "18215721889", "key": "click.org_onboarding_checklist.update_profile"}, {"experimentIds": [], "id": "18224360785", "key": "click.org_onboarding_checklist.dismiss"}, {"experimentIds": [], "id": "18234832286", "key": "submit.organization_activation.complete"}, {"experimentIds": [], "id": "18252392383", "key": "submit.org_repository.create"}, {"experimentIds": [], "id": "18257551537", "key": "submit.org_member_invitation.create"}, {"experimentIds": [], "id": "18259522260", "key": "submit.organization_profile.update"}, {"experimentIds": [], "id": "18564603625", "key": "view.classroom_select_organization"}, {"experimentIds": [], "id": "18568612016", "key": "click.classroom_sign_in_click"}, {"experimentIds": [], "id": "18572592540", "key": "view.classroom_name"}, {"experimentIds": [], "id": "18574203855", "key": "click.classroom_create_organization"}, {"experimentIds": [], "id": "18582053415", "key": "click.classroom_select_organization"}, {"experimentIds": [], "id": "18589463420", "key": "click.classroom_create_classroom"}, {"experimentIds": [], "id": "18591323364", "key": "click.classroom_create_first_classroom"}, {"experimentIds": [], "id": "18591652321", "key": "click.classroom_grant_access"}, {"experimentIds": [], "id": "18607131425", "key": "view.classroom_creation"}, {"experimentIds": [], "id": "18831680583", "key": "upgrade_account_plan"}, {"experimentIds": [], "id": "19064064515", "key": "click.signup"}, {"experimentIds": [], "id": "19075373687", "key": "click.view_account_billing_page"}, {"experimentIds": [], "id": "19077355841", "key": "click.dismiss_signup_prompt"}, {"experimentIds": [], "id": "19079713938", "key": "click.contact_sales"}, {"experimentIds": [], "id": "19120963070", "key": "click.compare_account_plans"}, {"experimentIds": [], "id": "19151690317", "key": "click.upgrade_account_cta"}, {"experimentIds": [], "id": "19424193129", "key": "click.open_accou

Now to recreate this graph in FSharp.FGl.ArrayAdjacencyGraph we simply have to create a vertex list and an edge list:
Once again we are using the data from [Correlation network](https://fslab.org/content/tutorials/009_correlation-network.html) .

In [2]:
#r "nuget: FSharp.FGL, 0.0.2"
#r "nuget: FSharp.FGL.ArrayAdjacencyGraph, 0.0.2"

open FSharp.FGL.ArrayAdjacencyGraph

// Creates a vertex list based on the cytoVertices
let vertexList =
    rawFrame.RowKeys
    |> Seq.toList
    |> List.indexed
    |> List.choose (fun (i,v) -> 
        let degree = 
            Matrix.getRow filteredNetwork i 
            |> Seq.filter ((<>) 0.)
            |> Seq.length
        
        if degree > 1 then 
            Some (i,v)
        else
            None
    )

// Creates an edge list based on the cytoEdges
let edgeList =
    let len = filteredNetwork.Dimensions |> fst
    [
        for i = 0 to len - 1 do
            for j = i + 1 to len - 1 do
                let v = filteredNetwork.[i,j]
                if v <> 0. then yield i,j,v
    ]

// Creates an ArrayAdjacencyGraph based on vertexList and edgeList
let startingGraph = 
    Graph.createOfEdgelist vertexList edgeList

Installed Packages FSharp.FGL, 0.0.2 FSharp.FGL.ArrayAdjacencyGraph, 0.0.2

## Modularity

One crucial factor in network science is the ability to represent large datasets in comprehensible graphs.
But when these graphs get to large, it is often very difficult to retrieve useful data from them without using some form of simplification.
One such method is the decomposition of networks into communities, sets of highly interconnected vertices.
By reducing the information of each of the vertices into these communities the size of the network can be reduced quite effectively.
The interdependence of the community-building vertices is often based on a functional module that the vertices belong to.
As such, the detection of communities is a really interesting factor in network science.
The Louvain-algorithm, published in [Blondel, Vincent D; Guillaume, Jean-Loup; Lambiotte, Renaud; Lefebvre, Etienne (9 October 2008). "Fast unfolding of communities in large networks". Journal of Statistical Mechanics: Theory and Experiment. 2008 (10): P10008. arXiv:0803.0476. Bibcode:2008JSMTE..10..008B. doi:10.1088/1742-5468/2008/10/P10008. S2CID 334423](https://doi.org/10.1088%2F1742-5468%2F2008%2F10%2FP10008) ,
is one of the possible algorithms for community detection and has been integrated into [FSharp.FGL](https://github.com/CSBiology/FSharp.FGL).

Here we apply this algorithm on our existing graph:

In [3]:
open ArrayAdjacencyGraph.Algorithms.Louvain.Louvain

// The graph after being grouped into communities
let louvainGraph = 
    louvain startingGraph 0.1 

new modularity= 0.4884290168


The Louvain algorithm reveals that the graph can be rationed into 34 communities. 
In the following steps, we color the communities that feature more than 5 members using Cyjs.Net.

In [19]:
#r "nuget: Plotly.NET, 4.0.0"
#r "nuget: Plotly.NET.Interactive, 4.0.0"

open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.Interactive

// Map that connected the community identifier with a color codex
let colorMap =
    
    // List of all communities that have more than 5 members
    let communitiesToColorList = 
        Vertices.getLabelList louvainGraph
        |> Array.map (snd)
        |> Array.countBy (fun x -> x)
        |> Array.choose(fun (m,count) -> 
            if count > 5 then
                Some m
            else
                None
            )
        |> List.ofArray

    // List of hexadecimal colors. 
    [
        "#A00976";
        "#D68A0C";
        "#13478D";
        "#9ACB0B";
        "#C249A1";
        "#FFC360";
        "#4B74AD";
        "#CCF35C";
        "#640048";
        "#855300";
        "#052858";
        "#5E7F00";
        "#CC0000";
        "#FFC0CB"

    ]
    |> List.map2(fun community color -> (community,color)) communitiesToColorList
    |> Map.ofList

// Map that maps the gene name to the color it is associated with because of its community
let geneNameToColor =
    
    let louvainLabelToColor= 
        Vertices.getLabelList louvainGraph
        |> Array.countBy (fun (gene,x) -> x)
        |> Array.choose(fun (m,count) -> 
            if count > 5 then
                Some m
            else
                None
            )
        |> Array.map(fun x -> (x,colorMap.Item (x)))
        |> Map.ofArray

    Vertices.getLabelList louvainGraph
    |> Array.map(fun (gene,label) -> 
        (gene,
            (if (Map.containsKey label louvainLabelToColor) then Map.find label louvainLabelToColor 
            else "#808080"))
    )
    |> Map.ofArray

// Table that showcases all communities, their member count and their connected color
let colorTable =
    let header = ["<b>Community</b>";"Number of members";"Color"]
    let rows = 
        Vertices.getLabelList louvainGraph
        |> Array.map (snd)
        |> Array.countBy (fun x -> x)
        |> Array.sortByDescending (fun (community,count) -> count)
        |> Array.map(fun (community,count) -> [(sprintf "%i" community);(sprintf "%i" count);(if (Map.containsKey community colorMap) then (colorMap.Item community) else "#808080")])
    let cellColor =
        Array.map(fun l -> 
            let color = Color.fromString (List.last l)
            [Color.fromString "transparent";Color.fromString "transparent";color]) rows
        |> Seq.transpose
        |> Seq.map Color.fromColors
        |> Color.fromColors
    Chart.Table(
        header,
        rows,
        HeaderOutlineColor = Color.fromString "black",
        CellsOutlineColor = Color.fromString "black",
        CellsOutlineWidth = 0.1,        
        CellsFillColor = cellColor
    )

colorTable


Installed Packages Plotly.NET, 4.0.0 Plotly.NET.Interactive, 4.0.0

<!-- Plotly chart will be drawn inside this DIV -->

In [20]:
// The styled vertices. The color of the vertices is based on their community. The size is based on the degree of this vertex, so that more heavily connected nodes are emphasized.
let cytoVertices2 =
    Array.map2 (fun v l -> (v,l)) (louvainGraph.GetVertices()) (Vertices.getLabelList louvainGraph)
    |> List.ofArray
    |> List.map(fun (v,(l,c)) ->
        let styling = [CyParam.label l;if (Map.containsKey c colorMap) then CyParam.color (colorMap.Item (c));CyParam.weight (sqrt (float (Vertices.degree v louvainGraph)) + 1. |> (*) 10.)]
        (Elements.node (string v) styling)

    )

// Creates an edge list based on the cytoEdges
let cytoEdges = 
    edgeList
    |> List.mapi (fun i (v1,v2,weight) -> 
        let styling = [CyParam.weight (0.2 * weight)]
        Elements.edge ("e" + string i) (string v1) (string v2) styling
    )

// Resulting cytograph
let cytoGraph2 = 

    CyGraph.initEmpty ()
    |> CyGraph.withElements cytoVertices2
    |> CyGraph.withElements cytoEdges
    |> CyGraph.withStyle "node" 
        [
            CyParam.shape "circle"
            CyParam.content =. CyParam.label
            CyParam.width =. CyParam.weight
            CyParam.height =. CyParam.weight
            CyParam.Text.Align.center
            CyParam.Border.color =. CyParam.color
            CyParam.Background.color =. CyParam.color
        ]
    |> CyGraph.withStyle "edge" 
        [
            CyParam.Line.color "#3D1244"
        ]
    |> CyGraph.withLayout (Layout.initCose (Layout.LayoutOptions.Cose(NodeOverlap = 400,ComponentSpacing = 100)))  

cytoGraph2
|> CyGraph.withSize (1300,1000)
|> Cyjs.NET.HTML.toEmbeddedHTML
|> DisplayFunctions.HTML

<!DOCTYPE html>

![Community detection](../../img/communityVsPCA.jpg)


## PCA - Principal component analysis

An entirely different approach to the graph analysis is given by the principal component analysis (PCA) . 
It is most commonly used for dimensional reduction and to build predictive models.
In our case we reduce the dimensions of experiments from the data we used to create the first graph.
More accurately, the PCA reduces the dataset by highlighting its biggest variances.
The PCA algorithm can be found in [FSharp.Stats](https://fslab.org/FSharp.Stats/).

In [21]:
let pcaData = 
    rows
    |> Matrix.transpose
    |> fun x ->
        let pcaComponents = ML.Unsupervised.PCA.compute (ML.Unsupervised.PCA.toAdjustCenter x) x

        let pComponent1 = pcaComponents.[0]
        let pComponent2 = pcaComponents.[1]
        let x = pComponent1.EigenVector 
        let y = pComponent2.EigenVector 
        
        let pcaCoordinates = Array.map2 (fun c1 c2 -> (c1,c2)) x y
        
        pcaCoordinates

Using the calculated data, we color each gene based on its coloring in the Louvain-Graph to showcase the differences between the community detection and the PCA.

In [25]:
// Creation of a pointchart based on the PCA data
let pcaChart = 
    let labelArray = rawFrame.RowKeys |> Array.ofSeq

    pcaData
    |> Array.mapi(fun i x -> 
        (labelArray.[i]),Chart.Point(
            [x],
            Text=labelArray.[i],
            MarkerColor=(
                if (Map.containsKey (labelArray.[i]) geneNameToColor) then
                    Color.fromHex(Map.find(labelArray.[i]) geneNameToColor)
                else
                    Color.fromString "gray"
                    ))|> Chart.withTraceInfo(Name=labelArray.[i]))
    |> Array.choose(fun (l,chart) -> if List.contains l (vertexList|>List.map snd) then Some chart else None)
    |> Chart.combine
    |> Chart.withYAxisStyle(TitleText="PC2")
    |> Chart.withXAxisStyle(TitleText="PC1")

pcaChart

<!-- Plotly chart will be drawn inside this DIV -->

While some communities from the Louvain analysis can be spotted in the PCA chart on behalf of their clustering, most of them are widespread, a sign for a high variance in the original dataset.
Using a clustering algorithm would lead to a much different communities than the PCA revealed.
A mathematical demonstration of this was deemed unnecessary, since most of the data points lie on top of another and would therefore be clustered together.

## Ontology Enrichment

Ontology enrichment is a method to identify overrepresented transcript/protein groups in data sets. 
The used annotation is based on the [AnnotationARC](https://github.com/CSBiology/ARCs/tree/main/AnnotationARC) .
and the calculation is done via [BioFSharp.Stats](https://github.com/CSBiology/BioFSharp) .
Here we try to find overrepresented protein functions in the Louvain communities to see if there is a functional modularity or if the communities are not that relevant, just like the PCA suggested. The result of the ontology enrichment can be seen in the following table. The headers are explained in the [BioFsharp doku](https://csbiology.github.io/BioFSharp//GSEA.html) .

In [28]:
#r "nuget: BioFSharp, 2.0.0-beta7"
#r "nuget: BioFSharp.Stats, 2.0.0-beta6"

open BioFSharp
open BioFSharp.Stats

// Maps the gene name to its molecular function based on the AnnotationARC
let geneNameToMolecularFunction =
    let d :Frame<string,string>=
        Frame.ReadCsv(@"../../files/ecoliGeneExpression.tsv",separators="\t")
        |> Frame.take 500
        |> Frame.indexRows "Key"

    let col :Series<string,string>=
        Frame.getCol "Gene ontology (molecular function)" d

    Seq.map2 (fun name f -> (name,f)) (col|> Series.keys) (col|> Series.values)
    |> Map.ofSeq

// List of all the community identifiers we want to use for ontology enrichment
let moduleNumbers = [|0..34|]

// Create the ontology terms on the gene data
let ontologyTerms =
    Vertices.getLabelList louvainGraph
    |> Array.map(fun (name,community) -> (name,[|1.223;2.123123|],community,(Map.find name geneNameToMolecularFunction)))
    |> Array.map (fun (name,data,modularityClass,annotation) ->
        let annotationProcessed = annotation.Replace ("| ","|")
        OntologyEnrichment.createOntologyItem name annotationProcessed modularityClass data
        |> OntologyEnrichment.splitMultipleAnnotationsBy '|' 
        )
    |> Seq.concat

// Run the ontology enrichment on every enty in ontologyTerms
let gseaResult =
    moduleNumbers
    |> Array.map (fun x ->
        x,OntologyEnrichment.CalcOverEnrichment x (Some 5) (Some 2) ontologyTerms
        )

// Return the enriched ontology entries 
let ontologyResult = 
    gseaResult
    |> Array.map (fun (moduleNumber,moduleEnrichment) ->
        let pvalues = moduleEnrichment |> Seq.map (fun x -> x.PValue)
        let pvaluesAdj = MultipleTesting.benjaminiHochbergFDR pvalues |> Array.ofSeq
        moduleEnrichment
        |> Seq.mapi (fun i x -> x,pvaluesAdj.[i])
        |> Seq.filter (fun (item,pvalAdj) -> item.PValue < 0.05) 
        |> Seq.sortByDescending (fun (item,pvalAdj) -> item.NumberOfDEsInBin) 
        |> Seq.map (fun (x,pvalAdj) ->
            //TotalUnivers was renamed to TotalUniverse
            [sprintf "%i" moduleNumber; x.OntologyTerm; sprintf "%i" x.TotalUnivers; sprintf "%i" x.TotalNumberOfDE; sprintf "%i" x.NumberInBin;sprintf "%i" x.NumberOfDEsInBin; sprintf "%f"x.PValue ;sprintf "%f" pvalAdj]
            
            )
        )
    |> Seq.choose(fun x -> if Seq.isEmpty x then None else Some x)
    |> Seq.map (fun x -> List.ofSeq x)
    |> List.ofSeq
    |> List.concat

// Build a Plotly.Net table of the ontology enrichment     
let ontologyTable =
    let header = ["<b>Community</b>";"OntologyTerm";"TotalUniverse";"TotalNumberOfDE";"NumberInBin";"NumberOfDEsInBin";"PValue";"pvalAdj"]
    let rows = 
        ontologyResult
    Chart.Table(header, rows)

ontologyTable
|> Chart.withSize(1300,1000)

Installed Packages BioFSharp, 2.0.0-beta7 BioFSharp.Stats, 2.0.0-beta6

<!-- Plotly chart will be drawn inside this DIV -->

From the table above I have chosen to show community 30 as an example of the differences between community detection and PCA.

In [37]:
let cytoVerticesOntology =
    Array.map2 (fun v l -> (v,l)) (louvainGraph.GetVertices()) (Vertices.getLabelList louvainGraph)
    |> List.ofArray
    |> List.map(fun (v,(l,c)) ->
        let styling = [CyParam.label l;if c=30 then CyParam.color (colorMap.Item (c));CyParam.weight (sqrt (float (louvainGraph.Degree v)) + 1. |> (*) 10.)]
        (Elements.node (string v) styling)

    )
let cytoGraphOntology = 

    CyGraph.initEmpty ()
    |> CyGraph.withElements cytoVerticesOntology
    |> CyGraph.withElements cytoEdges
    |> CyGraph.withStyle "node" 
        [
            CyParam.shape "circle"
            CyParam.content =. CyParam.label
            CyParam.width =. CyParam.weight
            CyParam.height =. CyParam.weight
            CyParam.Text.Align.center
            CyParam.Border.color =. CyParam.color
            CyParam.Background.color =. CyParam.color
        ]
    |> CyGraph.withStyle "edge" 
        [
            CyParam.Line.color "#3D1244"
        ]
    |> CyGraph.withLayout (Layout.initCose (Layout.LayoutOptions.Cose(NodeOverlap = 400,ComponentSpacing = 100)))  


![Community detection vs PCA](../../img/communityVsPCA.jpg)

While comparing community 30 with its PCA counterpart it becomes apparent 
that this functional group would not have been detected based on clustering of the PCA.
This does not mean that the PCA is without its merits, but merely show that 
the application of different approached to datasets can and will deliver different results.
The combination and comparison between different methods is an important step in data evaluation 
and should always be considered when thinking about your dataflow.

## Further reading

- [Blondel, Vincent D; Guillaume, Jean-Loup; Lambiotte, Renaud; Lefebvre, Etienne (9 October 2008). "Fast unfolding of communities in large networks". Journal of Statistical Mechanics: Theory and Experiment. 2008 (10): P10008. arXiv:0803.0476. Bibcode:2008JSMTE..10..008B. doi:10.1088/1742-5468/2008/10/P10008. S2CID 334423](https://doi.org/10.1088%2F1742-5468%2F2008%2F10%2FP10008)
- [Fslab](https://fslab.org)
- [FSharp.FGL](https://github.com/CSBiology/FSharp.FGL)

<html>

<head>

<script src="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min.js"></script>

</head>

<body> <style>#e29905fd9e2 { width: 1300px; height: 1000px; display: block } </style>

<div id="e29905fd9e2"></div>

<script type="text/javascript">



            var renderCyjs_92ed709a34b144a7a9df589dbe8efba7 = function() {

            var fsharpCyjsRequire = requirejs.config({context:'fsharp-cyjs',paths:{cyjs:'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min'}}) || require;

            fsharpCyjsRequire(['cyjs'], function(Cyjs) {



            var graphdata = {"container":document.getElementById('e29905fd9e2'),"elements":[{"data":{"id":"MS:1000031","label":"instrument model MS:1000031","weight":24}},{"data":{"id":"part_of","label":"instrument model part_of","weight":24}},{"data":{"id":"MS:1000121","label":"SCIEX instrument model MS:1000121","weight":24}},{"data":{"id":"MS:1000031","label":"SCIEX instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000122","label":"Bruker Daltonics instrument model MS:1000122","weight":24}},{"data":{"id":"MS:1000031","label":"Bruker Daltonics instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000124","label":"Shimadzu instrument model MS:1000124","weight":24}},{"data":{"id":"MS:1000031","label":"Shimadzu instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000126","label":"Waters instrument model MS:1000126","weight":24}},{"data":{"id":"MS:1000031","label":"Waters instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000483","label":"Thermo Fisher Scientific instrument model MS:1000483","weight":24}},{"data":{"id":"MS:1000031","label":"Thermo Fisher Scientific instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000488","label":"Hitachi instrument model MS:1000488","weight":24}},{"data":{"id":"MS:1000031","label":"Hitachi instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000489","label":"Varian instrument model MS:1000489","weight":24}},{"data":{"id":"MS:1000031","label":"Varian instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000490","label":"Agilent instrument model MS:1000490","weight":24}},{"data":{"id":"MS:1000031","label":"Agilent instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000491","label":"Dionex instrument model MS:1000491","weight":24}},{"data":{"id":"MS:1000031","label":"Dionex instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000495","label":"Applied Biosystems instrument model MS:1000495","weight":24}},{"data":{"id":"MS:1000031","label":"Applied Biosystems instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1001800","label":"LECO instrument model MS:1001800","weight":24}},{"data":{"id":"MS:1000031","label":"LECO instrument model MS:1000031","weight":24}},{"data":{"id":"e0","source":"MS:1000031","target":"part_of","weight":0.3}},{"data":{"id":"e1","source":"MS:1000121","target":"MS:1000031","weight":0.3}},{"data":{"id":"e2","source":"MS:1000122","target":"MS:1000031","weight":0.3}},{"data":{"id":"e3","source":"MS:1000124","target":"MS:1000031","weight":0.3}},{"data":{"id":"e4","source":"MS:1000126","target":"MS:1000031","weight":0.3}},{"data":{"id":"e5","source":"MS:1000483","target":"MS:1000031","weight":0.3}},{"data":{"id":"e6","source":"MS:1000488","target":"MS:1000031","weight":0.3}},{"data":{"id":"e7","source":"MS:1000489","target":"MS:1000031","weight":0.3}},{"data":{"id":"e8","source":"MS:1000490","target":"MS:1000031","weight":0.3}},{"data":{"id":"e9","source":"MS:1000491","target":"MS:1000031","weight":0.3}},{"data":{"id":"e10","source":"MS:1000495","target":"MS:1000031","weight":0.3}},{"data":{"id":"e11","source":"MS:1001800","target":"MS:1000031","weight":0.3}}],"style":[{"selector":"node","style":{"shape":"circle","content":"data(label)","border-color":"#A00975"}},{"selector":"edge","style":{"line-color":"#3D1244","curve-style":"bezier","target-arrow-shape":"triangle"}}],"layout":{"name":"breadthfirst"}}

            var cy = cytoscape( graphdata );

            cy.userZoomingEnabled( false );

            

});

            };

            if ((typeof(requirejs) !==  typeof(Function)) || (typeof(requirejs.config) !== typeof(Function))) {

                var script = document.createElement("script");

                script.setAttribute("src", "https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js");

                script.onload = function(){

                    renderCyjs_92ed709a34b144a7a9df589dbe8efba7();

                };

                document.getElementsByTagName("head")[0].appendChild(script);

            }

            else {

                renderCyjs_92ed709a34b144a7a9df589dbe8efba7();

            }

</script>

 </body>

</html>

### Task 7
- expand the graph using elements linked to `MS:1000121`
- triple elements can be appended by `Seq.append seq1 seq2`
- check out the graph

In [10]:
let msElements2 = getElements ms "MS:1000121"

let msVertices2 = getCytoVertices (Seq.append msElements msElements2)
let msEdges2 = getCytoEdges (Seq.append msElements msElements2)

cytoGraph msVertices2 msEdges2
|> CyGraph.withSize (1300,1000)

Error: input.fsx (1,19)-(1,30) typecheck error The value or constructor 'getElements' is not defined. Maybe you want one of the following:
   Elements
input.fsx (3,19)-(3,34) typecheck error The value or constructor 'getCytoVertices' is not defined. Maybe you want one of the following:
   getCvOfReplicates
   cytoVertices
input.fsx (4,16)-(4,28) typecheck error The value or constructor 'getCytoEdges' is not defined.
input.fsx (6,1)-(6,10) typecheck error This value is not a function and cannot be applied.

<html>

<head>

<script src="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min.js"></script>

</head>

<body> <style>#e03fc6d45ba { width: 1300px; height: 1000px; display: block } </style>

<div id="e03fc6d45ba"></div>

<script type="text/javascript">



            var renderCyjs_332a9a43714641d1b5a4e485ba54749b = function() {

            var fsharpCyjsRequire = requirejs.config({context:'fsharp-cyjs',paths:{cyjs:'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min'}}) || require;

            fsharpCyjsRequire(['cyjs'], function(Cyjs) {



            var graphdata = {"container":document.getElementById('e03fc6d45ba'),"elements":[{"data":{"id":"MS:1000031","label":"instrument model MS:1000031","weight":24}},{"data":{"id":"part_of","label":"instrument model part_of","weight":24}},{"data":{"id":"MS:1000121","label":"SCIEX instrument model MS:1000121","weight":24}},{"data":{"id":"MS:1000031","label":"SCIEX instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000122","label":"Bruker Daltonics instrument model MS:1000122","weight":24}},{"data":{"id":"MS:1000031","label":"Bruker Daltonics instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000124","label":"Shimadzu instrument model MS:1000124","weight":24}},{"data":{"id":"MS:1000031","label":"Shimadzu instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000126","label":"Waters instrument model MS:1000126","weight":24}},{"data":{"id":"MS:1000031","label":"Waters instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000483","label":"Thermo Fisher Scientific instrument model MS:1000483","weight":24}},{"data":{"id":"MS:1000031","label":"Thermo Fisher Scientific instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000488","label":"Hitachi instrument model MS:1000488","weight":24}},{"data":{"id":"MS:1000031","label":"Hitachi instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000489","label":"Varian instrument model MS:1000489","weight":24}},{"data":{"id":"MS:1000031","label":"Varian instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000490","label":"Agilent instrument model MS:1000490","weight":24}},{"data":{"id":"MS:1000031","label":"Agilent instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000491","label":"Dionex instrument model MS:1000491","weight":24}},{"data":{"id":"MS:1000031","label":"Dionex instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000495","label":"Applied Biosystems instrument model MS:1000495","weight":24}},{"data":{"id":"MS:1000031","label":"Applied Biosystems instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1001800","label":"LECO instrument model MS:1001800","weight":24}},{"data":{"id":"MS:1000031","label":"LECO instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000121","label":"SCIEX instrument model MS:1000121","weight":24}},{"data":{"id":"MS:1000031","label":"SCIEX instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000139","label":"4000 QTRAP MS:1000139","weight":24}},{"data":{"id":"MS:1000121","label":"4000 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1000143","label":"API 150EX MS:1000143","weight":24}},{"data":{"id":"MS:1000121","label":"API 150EX MS:1000121","weight":24}},{"data":{"id":"MS:1000144","label":"API 150EX Prep MS:1000144","weight":24}},{"data":{"id":"MS:1000121","label":"API 150EX Prep MS:1000121","weight":24}},{"data":{"id":"MS:1000145","label":"API 2000 MS:1000145","weight":24}},{"data":{"id":"MS:1000121","label":"API 2000 MS:1000121","weight":24}},{"data":{"id":"MS:1000146","label":"API 3000 MS:1000146","weight":24}},{"data":{"id":"MS:1000121","label":"API 3000 MS:1000121","weight":24}},{"data":{"id":"MS:1000147","label":"API 4000 MS:1000147","weight":24}},{"data":{"id":"MS:1000121","label":"API 4000 MS:1000121","weight":24}},{"data":{"id":"MS:1000186","label":"proteomics solution 1 MS:1000186","weight":24}},{"data":{"id":"MS:1000121","label":"proteomics solution 1 MS:1000121","weight":24}},{"data":{"id":"MS:1000187","label":"Q TRAP MS:1000187","weight":24}},{"data":{"id":"MS:1000121","label":"Q TRAP MS:1000121","weight":24}},{"data":{"id":"MS:1000190","label":"QSTAR MS:1000190","weight":24}},{"data":{"id":"MS:1000121","label":"QSTAR MS:1000121","weight":24}},{"data":{"id":"MS:1000194","label":"SymBiot I MS:1000194","weight":24}},{"data":{"id":"MS:1000121","label":"SymBiot I MS:1000121","weight":24}},{"data":{"id":"MS:1000195","label":"SymBiot XVI MS:1000195","weight":24}},{"data":{"id":"MS:1000121","label":"SymBiot XVI MS:1000121","weight":24}},{"data":{"id":"MS:1000651","label":"3200 QTRAP MS:1000651","weight":24}},{"data":{"id":"MS:1000121","label":"3200 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1000652","label":"4800 Plus MALDI TOF/TOF MS:1000652","weight":24}},{"data":{"id":"MS:1000121","label":"4800 Plus MALDI TOF/TOF MS:1000121","weight":24}},{"data":{"id":"MS:1000653","label":"API 3200 MS:1000653","weight":24}},{"data":{"id":"MS:1000121","label":"API 3200 MS:1000121","weight":24}},{"data":{"id":"MS:1000654","label":"API 5000 MS:1000654","weight":24}},{"data":{"id":"MS:1000121","label":"API 5000 MS:1000121","weight":24}},{"data":{"id":"MS:1000655","label":"QSTAR Elite MS:1000655","weight":24}},{"data":{"id":"MS:1000121","label":"QSTAR Elite MS:1000121","weight":24}},{"data":{"id":"MS:1000656","label":"QSTAR Pulsar MS:1000656","weight":24}},{"data":{"id":"MS:1000121","label":"QSTAR Pulsar MS:1000121","weight":24}},{"data":{"id":"MS:1000657","label":"QSTAR XL MS:1000657","weight":24}},{"data":{"id":"MS:1000121","label":"QSTAR XL MS:1000121","weight":24}},{"data":{"id":"MS:1000870","label":"4000 QTRAP MS:1000870","weight":24}},{"data":{"id":"MS:1000121","label":"4000 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1000931","label":"QTRAP 5500 MS:1000931","weight":24}},{"data":{"id":"MS:1000121","label":"QTRAP 5500 MS:1000121","weight":24}},{"data":{"id":"MS:1000932","label":"TripleTOF 5600 MS:1000932","weight":24}},{"data":{"id":"MS:1000121","label":"TripleTOF 5600 MS:1000121","weight":24}},{"data":{"id":"MS:1001482","label":"5800 TOF/TOF MS:1001482","weight":24}},{"data":{"id":"MS:1000121","label":"5800 TOF/TOF MS:1000121","weight":24}},{"data":{"id":"MS:1002533","label":"TripleTOF 6600 MS:1002533","weight":24}},{"data":{"id":"MS:1000121","label":"TripleTOF 6600 MS:1000121","weight":24}},{"data":{"id":"MS:1002577","label":"2000 QTRAP MS:1002577","weight":24}},{"data":{"id":"MS:1000121","label":"2000 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1002578","label":"2500 QTRAP MS:1002578","weight":24}},{"data":{"id":"MS:1000121","label":"2500 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1002579","label":"3500 QTRAP MS:1002579","weight":24}},{"data":{"id":"MS:1000121","label":"3500 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1002580","label":"QTRAP 4500 MS:1002580","weight":24}},{"data":{"id":"MS:1000121","label":"QTRAP 4500 MS:1000121","weight":24}},{"data":{"id":"MS:1002581","label":"QTRAP 6500 MS:1002581","weight":24}},{"data":{"id":"MS:1000121","label":"QTRAP 6500 MS:1000121","weight":24}},{"data":{"id":"MS:1002582","label":"QTRAP 6500+ MS:1002582","weight":24}},{"data":{"id":"MS:1000121","label":"QTRAP 6500+ MS:1000121","weight":24}},{"data":{"id":"MS:1002583","label":"TripleTOF 4600 MS:1002583","weight":24}},{"data":{"id":"MS:1000121","label":"TripleTOF 4600 MS:1000121","weight":24}},{"data":{"id":"MS:1002584","label":"TripleTOF 5600+ MS:1002584","weight":24}},{"data":{"id":"MS:1000121","label":"TripleTOF 5600+ MS:1000121","weight":24}},{"data":{"id":"MS:1002585","label":"API 100 MS:1002585","weight":24}},{"data":{"id":"MS:1000121","label":"API 100 MS:1000121","weight":24}},{"data":{"id":"MS:1002586","label":"API 100LC MS:1002586","weight":24}},{"data":{"id":"MS:1000121","label":"API 100LC MS:1000121","weight":24}},{"data":{"id":"MS:1002587","label":"API 165 MS:1002587","weight":24}},{"data":{"id":"MS:1000121","label":"API 165 MS:1000121","weight":24}},{"data":{"id":"MS:1002588","label":"API 300 MS:1002588","weight":24}},{"data":{"id":"MS:1000121","label":"API 300 MS:1000121","weight":24}},{"data":{"id":"MS:1002589","label":"API 350 MS:1002589","weight":24}},{"data":{"id":"MS:1000121","label":"API 350 MS:1000121","weight":24}},{"data":{"id":"MS:1002590","label":"API 365 MS:1002590","weight":24}},{"data":{"id":"MS:1000121","label":"API 365 MS:1000121","weight":24}},{"data":{"id":"MS:1002591","label":"Triple Quad 3500 MS:1002591","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 3500 MS:1000121","weight":24}},{"data":{"id":"MS:1002592","label":"Triple Quad 4500 MS:1002592","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 4500 MS:1000121","weight":24}},{"data":{"id":"MS:1002593","label":"Triple Quad 5500 MS:1002593","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 5500 MS:1000121","weight":24}},{"data":{"id":"MS:1002594","label":"Triple Quad 6500 MS:1002594","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 6500 MS:1000121","weight":24}},{"data":{"id":"MS:1002595","label":"Triple Quad 6500+ MS:1002595","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 6500+ MS:1000121","weight":24}},{"data":{"id":"MS:1002674","label":"X500R QTOF MS:1002674","weight":24}},{"data":{"id":"MS:1000121","label":"X500R QTOF MS:1000121","weight":24}},{"data":{"id":"MS:1003144","label":"Triple Quad 7500 MS:1003144","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 7500 MS:1000121","weight":24}},{"data":{"id":"MS:1003293","label":"ZenoTOF 7600 MS:1003293","weight":24}},{"data":{"id":"MS:1000121","label":"ZenoTOF 7600 MS:1000121","weight":24}},{"data":{"id":"e0","source":"MS:1000031","target":"part_of","weight":0.3}},{"data":{"id":"e1","source":"MS:1000121","target":"MS:1000031","weight":0.3}},{"data":{"id":"e2","source":"MS:1000122","target":"MS:1000031","weight":0.3}},{"data":{"id":"e3","source":"MS:1000124","target":"MS:1000031","weight":0.3}},{"data":{"id":"e4","source":"MS:1000126","target":"MS:1000031","weight":0.3}},{"data":{"id":"e5","source":"MS:1000483","target":"MS:1000031","weight":0.3}},{"data":{"id":"e6","source":"MS:1000488","target":"MS:1000031","weight":0.3}},{"data":{"id":"e7","source":"MS:1000489","target":"MS:1000031","weight":0.3}},{"data":{"id":"e8","source":"MS:1000490","target":"MS:1000031","weight":0.3}},{"data":{"id":"e9","source":"MS:1000491","target":"MS:1000031","weight":0.3}},{"data":{"id":"e10","source":"MS:1000495","target":"MS:1000031","weight":0.3}},{"data":{"id":"e11","source":"MS:1001800","target":"MS:1000031","weight":0.3}},{"data":{"id":"e12","source":"MS:1000139","target":"MS:1000121","weight":0.3}},{"data":{"id":"e13","source":"MS:1000143","target":"MS:1000121","weight":0.3}},{"data":{"id":"e14","source":"MS:1000144","target":"MS:1000121","weight":0.3}},{"data":{"id":"e15","source":"MS:1000145","target":"MS:1000121","weight":0.3}},{"data":{"id":"e16","source":"MS:1000146","target":"MS:1000121","weight":0.3}},{"data":{"id":"e17","source":"MS:1000147","target":"MS:1000121","weight":0.3}},{"data":{"id":"e18","source":"MS:1000186","target":"MS:1000121","weight":0.3}},{"data":{"id":"e19","source":"MS:1000187","target":"MS:1000121","weight":0.3}},{"data":{"id":"e20","source":"MS:1000190","target":"MS:1000121","weight":0.3}},{"data":{"id":"e21","source":"MS:1000194","target":"MS:1000121","weight":0.3}},{"data":{"id":"e22","source":"MS:1000195","target":"MS:1000121","weight":0.3}},{"data":{"id":"e23","source":"MS:1000651","target":"MS:1000121","weight":0.3}},{"data":{"id":"e24","source":"MS:1000652","target":"MS:1000121","weight":0.3}},{"data":{"id":"e25","source":"MS:1000653","target":"MS:1000121","weight":0.3}},{"data":{"id":"e26","source":"MS:1000654","target":"MS:1000121","weight":0.3}},{"data":{"id":"e27","source":"MS:1000655","target":"MS:1000121","weight":0.3}},{"data":{"id":"e28","source":"MS:1000656","target":"MS:1000121","weight":0.3}},{"data":{"id":"e29","source":"MS:1000657","target":"MS:1000121","weight":0.3}},{"data":{"id":"e30","source":"MS:1000870","target":"MS:1000121","weight":0.3}},{"data":{"id":"e31","source":"MS:1000931","target":"MS:1000121","weight":0.3}},{"data":{"id":"e32","source":"MS:1000932","target":"MS:1000121","weight":0.3}},{"data":{"id":"e33","source":"MS:1001482","target":"MS:1000121","weight":0.3}},{"data":{"id":"e34","source":"MS:1002533","target":"MS:1000121","weight":0.3}},{"data":{"id":"e35","source":"MS:1002577","target":"MS:1000121","weight":0.3}},{"data":{"id":"e36","source":"MS:1002578","target":"MS:1000121","weight":0.3}},{"data":{"id":"e37","source":"MS:1002579","target":"MS:1000121","weight":0.3}},{"data":{"id":"e38","source":"MS:1002580","target":"MS:1000121","weight":0.3}},{"data":{"id":"e39","source":"MS:1002581","target":"MS:1000121","weight":0.3}},{"data":{"id":"e40","source":"MS:1002582","target":"MS:1000121","weight":0.3}},{"data":{"id":"e41","source":"MS:1002583","target":"MS:1000121","weight":0.3}},{"data":{"id":"e42","source":"MS:1002584","target":"MS:1000121","weight":0.3}},{"data":{"id":"e43","source":"MS:1002585","target":"MS:1000121","weight":0.3}},{"data":{"id":"e44","source":"MS:1002586","target":"MS:1000121","weight":0.3}},{"data":{"id":"e45","source":"MS:1002587","target":"MS:1000121","weight":0.3}},{"data":{"id":"e46","source":"MS:1002588","target":"MS:1000121","weight":0.3}},{"data":{"id":"e47","source":"MS:1002589","target":"MS:1000121","weight":0.3}},{"data":{"id":"e48","source":"MS:1002590","target":"MS:1000121","weight":0.3}},{"data":{"id":"e49","source":"MS:1002591","target":"MS:1000121","weight":0.3}},{"data":{"id":"e50","source":"MS:1002592","target":"MS:1000121","weight":0.3}},{"data":{"id":"e51","source":"MS:1002593","target":"MS:1000121","weight":0.3}},{"data":{"id":"e52","source":"MS:1002594","target":"MS:1000121","weight":0.3}},{"data":{"id":"e53","source":"MS:1002595","target":"MS:1000121","weight":0.3}},{"data":{"id":"e54","source":"MS:1002674","target":"MS:1000121","weight":0.3}},{"data":{"id":"e55","source":"MS:1003144","target":"MS:1000121","weight":0.3}},{"data":{"id":"e56","source":"MS:1003293","target":"MS:1000121","weight":0.3}}],"style":[{"selector":"node","style":{"shape":"circle","content":"data(label)","border-color":"#A00975"}},{"selector":"edge","style":{"line-color":"#3D1244","curve-style":"bezier","target-arrow-shape":"triangle"}}],"layout":{"name":"breadthfirst"}}

            var cy = cytoscape( graphdata );

            cy.userZoomingEnabled( false );

            

});

            };

            if ((typeof(requirejs) !==  typeof(Function)) || (typeof(requirejs.config) !== typeof(Function))) {

                var script = document.createElement("script");

                script.setAttribute("src", "https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js");

                script.onload = function(){

                    renderCyjs_332a9a43714641d1b5a4e485ba54749b();

                };

                document.getElementsByTagName("head")[0].appendChild(script);

            }

            else {

                renderCyjs_332a9a43714641d1b5a4e485ba54749b();

            }

</script>

 </body>

</html>